# Latest Product Table

- p_id from **1 to 100** already present in  main table, it can be updated and deleted
- p_id from **101 to 200** can be inserted in to main table and after inserting it into main table it could also be deleted


### Libraries Used

In [1]:
import datetime
import os

# Main entry point for DataFrame and SQL functionality.
from pyspark.sql import SparkSession
# Start SPARK Session
spark = SparkSession.builder.appName('Basics').getOrCreate()

from pyspark.sql.functions import *
from pyspark.sql.types import *


### Read the Main Table

In [2]:
# Read from main_table
Table = spark.read.format('csv').options(
    header=True, inferschema=True).load(
        "/home/bluepi/Downloads/Update/product_info/main table/main_table_new.csv")

# Add record type to main table
Table_new = Table.withColumn('record_type', lit("A"))

# Write in main_table folder
path = "/home/bluepi/Downloads/Update/Updated Product/Latest Product/"
try:
    Table_new.write.format('csv').save(
        os.path.join(path, 'main_table'), header=True)
except:
    Table_new.write.mode('overwrite').format('csv').save(
        os.path.join(path, 'main_table'), header=True)


In [3]:
# GFG
class my_dictionary(dict):

    # __init__ function
    def __init__(self):
        self = dict()

    # Function to add key:value
    def add(self, key, value):
        self[key] = value

### Main CODE

In [4]:
total_products_updated = my_dictionary()
total_products_inserted = my_dictionary()
total_products_deleted = my_dictionary()


for i in range(10, 0, -1):

    data_schema = [
        StructField('p_id', IntegerType(), True),
        StructField('p_name', StringType(), True),
        StructField('price', IntegerType(), True),
        StructField('date_timestamp', TimestampType(), True),
        StructField('record_type', StringType(), True)
    ]
    # Schema which we are accepting
    final_struc = StructType(fields=data_schema)

    # In this try-catch block we are only taking the previous day data, but initially there is
    # no previous day, therefore we are taking main table at begining
    try:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", t)
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass

        mainTable = spark.read.csv(path1, header=True, schema=final_struc)
    except:
        path1 = os.path.join(
            "/home/bluepi/Downloads/Update/Updated Product/Latest Product/", "main_table")
        try:
            os.remove(os.path.join(path1, '_SUCCESS'))
        except:
            pass
        mainTable = spark.read.csv(path1, header=True, schema=final_struc)

    print("\n File read from --")
    print(path1, end='\n')

    # Address to the product_info folder
    address = "/home/bluepi/Downloads/Update/product_info/"
    previous_day = (datetime.datetime.today() -
                    datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    print("\n\t\ Date ----> "+previous_day+"\n")

    # before_insert = str(mainTable.count())
    print("Main_Table Count ----> " + str(mainTable.count()))

    # Address to the Previous Day folder
    new_address = address + previous_day
#     print("\nNew Address to read the folder ---->"+new_address)

# ###################################################################################################

    # Read the Previous Day folder
    per_day_data = spark.read.format('csv') \
        .options(header=True, inferschema=True) \
        .load(new_address)

    # Get the list of p_id which we have to inserted in the main table
    to_be_inserted = per_day_data.filter("record_type == 'I' ").collect()
    p_id_list_I = [i.p_id for i in to_be_inserted]

    # Directly append new Inserted products
    mainTable_I_inserted = mainTable.union(
        per_day_data.filter("record_type == 'I' "))

    after_insert = str(mainTable_I_inserted.count())
    total_insert = str(per_day_data.filter("record_type == 'I' ").count())

    print("After Inserting----> " + after_insert)
    print("Total \"Inserted I\"----> " + total_insert)

    total_products_inserted.add(previous_day, p_id_list_I)
    p_id_list_I.sort()
    print(" ".join(map(str, p_id_list_I)))
    print("\n")

# ###################################################################################################

# Created a new DataFrame of records to be updated
    from_per_day_data_U = per_day_data.filter("record_type == 'U' ")
    
# Get the list of p_id which we have to update taken from dated folders
    from_per_day_data_U_list = from_per_day_data_U.select("p_id").collect()

# List comprehension
    p_id_list_U = [i.p_id for i in from_per_day_data_U_list]
    p_id_list_U.sort()
    print("TO Be Update\"per_day_data\"")
    print(" ".join(map(str, p_id_list_U)))
#     print(p_id_list_U)

    total_products_updated.add(previous_day, p_id_list_U)

    total_update = str(len((p_id_list_U)))
    print("Update Count ------>"+total_update)
    print("\n")
    
# Get the products which we have to update, present in Main Table
    from_mainTable_U = mainTable_I_inserted.filter(
        col('p_id').isin(p_id_list_U))

    # Performed Union operation on the above tables
    mT_and_pDD_union = from_mainTable_U.union(from_per_day_data_U)
#     mT_and_pDD_union.orderBy(mT_and_pDD_union.p_id).show()

    # Performed GroupBy operation on P_ID and take the latest date only
    #    Rename the columns
    for_join_mT_and_pDD = mT_and_pDD_union.groupBy("p_id").agg(
        {"date_timestamp": "max"}).withColumnRenamed("max(date_timestamp)", "date_timestamp_1")
    for_join_mT_and_pDD = for_join_mT_and_pDD.withColumnRenamed(
        "p_id", "p_id_1")
#     for_join_mT_and_pDD.show()

    # Performed Join operation to pick only latest updates only
    joined = mT_and_pDD_union.join(for_join_mT_and_pDD, (
        mT_and_pDD_union.p_id == for_join_mT_and_pDD.p_id_1) & (
            mT_and_pDD_union.date_timestamp == for_join_mT_and_pDD.date_timestamp_1), 'inner')

    joined = joined.select(
        ['p_id', 'p_name', 'price', 'date_timestamp', 'record_type'])
#     joined.count()

    # First remove the p_id from Main_Table which we have to updated
    mainTable_U_updated = mainTable_I_inserted.filter(
        ~col('p_id').isin(p_id_list_U))
#     mainTable_U_updated.orderBy("p_id").count()

    # Then Add the Updated P_ID to the Main_Table
    mainTable_U_updated_new = mainTable_U_updated.union(joined)
#     after_update = str(mainTable_U_updated_new.orderBy("p_id").count())
#     mainTable_U_updated_new.orderBy("p_id").count()

# ###################################################################################################

    # Get the list of p_id which we have to delete
    to_be_deleted = per_day_data.filter("record_type == 'D' ").collect()
    p_id_list_D = [i.p_id for i in to_be_deleted]

    total_products_deleted.add(previous_day, p_id_list_D)

#     print("\nList of p_id which we have to deleted taken from \"per_day_data\"")
#     print(p_id_list_D)
    total_deleted = str(len((p_id_list_D)))
    p_id_list_D.sort()
    print("\nTotal Deleted ------>"+str(len((p_id_list_D))))
    print(" ".join(map(str, p_id_list_D)))

    # Remove the deleted p_id from main_table
    mainTable_D_deleted = mainTable_U_updated_new.filter(
        ~col('p_id').isin(p_id_list_D))

    after_delete = str(mainTable_D_deleted.count())
    print("After Deleting Count---->"+after_delete)

# ###################################################################################################

    # This writes the DF in different files becaues of parallism
    t = "main_table"+str(previous_day)
#     print(t,end="\n")

#     try:
    mainTable_D_deleted.write.format('csv').save(
            os.path.join(path, t), header=True)
#         print("\n Main Table Stored (overwritten-NO)")
#     except:
#         mainTable_D_deleted.write.mode('overwrite').format(
#             'csv').save(os.path.join(path, t), header=True)
#         print("\n Main Table Stored (overwritten-YES)")

    print("\n Main Table Stored ")
    print(os.path.join(path, t))

    print("\n***************************************************************************************\n")
#     input("Press Enter to continue...")


 File read from --
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table

	\ Date ----> 30-03-2020

Main_Table Count ----> 100
After Inserting----> 109
Total "Inserted I"----> 9
126 127 132 138 143 166 170 175 195


TO Be Update"per_day_data"
1 1 2 10 18 18 33 43 47 49 50 56 56 64 75 97 99 170 175
Update Count ------>19



Total Deleted ------>3
10 23 93
After Deleting Count---->106

 Main Table Stored 
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table30-03-2020

***************************************************************************************


 File read from --
/home/bluepi/Downloads/Update/Updated Product/Latest Product/main_table30-03-2020

	\ Date ----> 31-03-2020

Main_Table Count ----> 106
After Inserting----> 114
Total "Inserted I"----> 8
102 110 130 140 142 154 190 199


TO Be Update"per_day_data"
6 8 28 41 44 62 65 67 67 73 86 92 126 138 154 199
Update Count ------>16



Total Deleted ------>7
41 59 79 80 82 95 110
After Deletin

### Products Updated more than once in one day

In [5]:
l = mT_and_pDD_union.groupBy("p_id").agg({'p_id':'count'}).filter("count(p_id)>2").select('p_id').collect()
l1 = [ i.p_id for i in l]
mT_and_pDD_union.filter( col('p_id').isin(l1) ).show()

+----+-------+-----+-------------------+-----------+
|p_id| p_name|price|     date_timestamp|record_type|
+----+-------+-----+-------------------+-----------+
|  27|Konklux|  348|2020-01-31 12:12:08|          A|
|  27|Konklux|  826|2020-04-07 09:49:08|          U|
|  27|Konklux|   88|2020-04-07 15:29:30|          U|
+----+-------+-----+-------------------+-----------+



### Final Main Table

In [6]:
mainTable_D_deleted.filter( col('p_id').isin(l1) ).show()

+----+-------+-----+-------------------+-----------+
|p_id| p_name|price|     date_timestamp|record_type|
+----+-------+-----+-------------------+-----------+
|  27|Konklux|   88|2020-04-07 15:29:30|          U|
+----+-------+-----+-------------------+-----------+



In [7]:
import pprint
# prints the formatted representation of PrettyPrinter object
pp = pprint.PrettyPrinter(indent=4)

for i in range(10,0,-1) :
    previous_day = (datetime.datetime.today() - datetime.timedelta(days=i)).strftime('%d-%m-%Y')
    pp.pprint(previous_day)
    print("inserted :\t")
    print(' '.join(map(str, total_products_inserted.get(previous_day))))
#     pp.pprint(total_products_inserted.get(previous_day))
    print("updated :\t")
    print(' '.join(map(str, total_products_updated.get(previous_day))))
#     pp.pprint(total_products_updated.get(previous_day))
    print("deleted :\t")
    print(' '.join(map(str, total_products_deleted.get(previous_day))))
#     pp.pprint(total_products_deleted.get(previous_day))
    print("\n")

'30-03-2020'
inserted :	
126 127 132 138 143 166 170 175 195
updated :	
1 1 2 10 18 18 33 43 47 49 50 56 56 64 75 97 99 170 175
deleted :	
10 23 93


'31-03-2020'
inserted :	
102 110 130 140 142 154 190 199
updated :	
6 8 28 41 44 62 65 67 67 73 86 92 126 138 154 199
deleted :	
41 59 79 80 82 95 110


'01-04-2020'
inserted :	
103 112 114 119 136 163 177 193
updated :	
7 11 14 17 22 24 24 34 35 40 44 55 60 66 73 76 81 84 85 88 90 100 102 112 143
deleted :	
15 26 31 50 57 66 90 138 142


'02-04-2020'
inserted :	
113 139 153 161 168 176 179
updated :	
1 19 29 40 45 61 72 81 84 85 88 92 119 132 175
deleted :	
36 51 60 69 75 100 140 161


'03-04-2020'
inserted :	
105 107 120 128 131 146 150 158 165 167 182 185
updated :	
1 14 38 44 48 56 62 73 87 91 96 107 119 119 127 199
deleted :	
28 34 64 143 161 176


'04-04-2020'
inserted :	
101 109 123 125 147 159 164 184 188
updated :	
3 13 14 19 22 24 58 67 70 88 89 89 92 97 99 102 102 107 127 131 132 177 185 190 195
deleted :	
7 43 68 114 130


'05